In [1]:
import random
import string

A_opts = ["G", "C", "T"]
C_opts = ["A", "G", "T"]
G_opts = ["A", "C", "T"]
T_opts = ["A", "C", "G"]

# Method to change N characters from a string with random characters.
def randomlyChangeNChar(word, n):
    length = len(word)
    word = list(word) #list of every char in the str
    k = random.sample(range(0,length),n) #list of n random indices i.e. chars in the list of str chars
    #print("The random indices are "+str(k))
    for index in k:
        # This will replace the chars at the specified index with the randomly-generated chars
        if word[index] == "A":
            word[index] = random.choice(A_opts)
        elif word[index] == "C":
            word[index] = random.choice(C_opts)
        elif word[index] == "G":
            word[index] = random.choice(G_opts)
        elif word[index] == "T":
            word[index] = random.choice(T_opts)

    # Finally print the string in the modified format.
    result = "" . join(word)
    #print("" . join(word))
    return result

# attribution: https://stackoverflow.com/questions/40425206/how-to-randomly-change-n-characters-in-a-string

In [2]:
import pandas as pd
df = pd.read_csv('./Josh_codon_project/codon_avgs.csv')
codon_avgs_dict  = dict(df.values)

def getCSC(Seq):
    # separate sequence by codon, and put into list
    CodonList = []
    for x in range(0, len(Seq), 3):
        # if it's a full codon
        if len(Seq[x:x+3]) == 3:          
            CodonList.append(Seq[x:x+3].upper())
    
    # for each codon, add to sumScore it's CSC score and then find the avg across the whole sequence
    sumScore = 0
    numstops = 0
    for codon in CodonList:
        if codon in codon_avgs_dict:
            score = codon_avgs_dict[codon]
            sumScore += score
        else: #is a stop codon (still counted as codon in CodonList, but doesnt have CSC score, so dont include in avg)
            numstops += 1
    avgScore = sumScore / (len(CodonList) - numstops)

    return avgScore

In [83]:
#read in csv file containing sequences you wish to randomize & calculate avgs for
df = pd.read_csv('./Random/MLN.csv')
df.head()

,SeqName,V1,VDJ seq,TotalMuts,Germline,"Average CSC of ""VDJ seq""","Average CSC of ""germline""",Average of CSCs for 10 Germline+RandomMut,StdDev of CSCs for 10 Germline+RandomMut
0,JM12-PM1-MLN10L_2391_H2,IGHV2-70*19,CAGGTCACCCTGAGGGAGTCCGGTCCTGCACTAGTAAAGCCCCCCC...,75,CAGGTCACCTTGAGGGAGTCTGGTCCTGCGCTGGTGAAACCCACAC...,0.025991,0.023579,NaN,NaN
1,JM12-PM1-MLN10L_3148_H1,IGHV2-70*19,CAGGTCACCCTGAGGGAGTCCGGTCCTGCACTAGTAAAGCCCCCCC...,75,CAGGTCACCTTGAGGGAGTCTGGTCCTGCGCTGGTGAAACCCACAC...,0.025991,0.023579,NaN,NaN
2,JM12-PM1-MLN10L_950_H1,IGHV2-70*19,CAGGTCACCCTGAGGGAGTCCGGTCCTGCACTAGTAAAGCCCCCCC...,75,CAGGTCACCTTGAGGGAGTCTGGTCCTGCGCTGGTGAAACCCACAC...,0.025991,0.023579,NaN,NaN
3,JM12-PM1-MLN10R_1159_H2,IGHV4-59*02,CAGGTGCAGCTGCAAGAGTCGGGCCCAGGACTGCTGAAGCCTGCGG...,70,CAGGTGCAGCTGCAGGAGTCGGGCCCAGGACTGGTGAAGCCTTCGG...,0.020797,0.025760,NaN,NaN
4,JM12-PM1-MLN10R_1902_H2,IGHV1-69*04,CAGATCCAGTTGCTACAGTCTGGGCCTGAGGTGAAGAAGCCTGGGT...,67,CAGGTCCAGCTGGTGCAGTCTGGGGCTGAGGTGAAGAAGCCTGGGT...,0.017293,0.025106,NaN,NaN


In [84]:
import statistics as st

avg_csc = []
stddev_csc = []
random_muts = []
random_muts_cscs = []

#for each row of csv file (which is one sequence)
for index, row in df.iterrows():
    #initialize variables and storage
    word = row['Germline']
    #print(word)
    n = row['TotalMuts']
    #print(n)
    seq_storage = []
    csc_storage = []
    
    #generate 10 random mutations
    for i in range(10):
        seq_storage.append(randomlyChangeNChar(word, n)) 
    random_muts.append(seq_storage)
    
    #compute CSC values of those 10
    for j in range(len(seq_storage)):
        csc_storage.append(getCSC(seq_storage[j]))
    random_muts_cscs.append(csc_storage)
    
    #compute average and standard deviation   
    avg_csc.append(st.mean(csc_storage))
    stddev_csc.append(st.stdev(csc_storage))

df['Average of CSCs for 10 Germline+RandomMut'] = avg_csc
df['StdDev of CSCs for 10 Germline+RandomMut'] = stddev_csc
df['Random Variants'] = random_muts
df['Random Variant CSC Values'] = random_muts_cscs

df.head()

,SeqName,V1,VDJ seq,TotalMuts,Germline,"Average CSC of ""VDJ seq""","Average CSC of ""germline""",Average of CSCs for 10 Germline+RandomMut,StdDev of CSCs for 10 Germline+RandomMut,Random Variants,Random Variant CSC Values
0,JM12-PM1-MLN10L_2391_H2,IGHV2-70*19,CAGGTCACCCTGAGGGAGTCCGGTCCTGCACTAGTAAAGCCCCCCC...,75,CAGGTCACCTTGAGGGAGTCTGGTCCTGCGCTGGTGAAACCCACAC...,0.025991,0.023579,0.012770,0.003060,[CAGGTGGCCAACAGGGAATCTGGTTCTTCGCTGGTGCAATCCACA...,"[0.015570694132231408, 0.011325039024793388, 0..."
1,JM12-PM1-MLN10L_3148_H1,IGHV2-70*19,CAGGTCACCCTGAGGGAGTCCGGTCCTGCACTAGTAAAGCCCCCCC...,75,CAGGTCACCTTGAGGGAGTCTGGTCCTGCGCTGGTGAAACCCACAC...,0.025991,0.023579,0.014584,0.004509,[CCGGTGACCTTGGGGGAGTCTGGTTCTGCACTGGTGAAGCCCACA...,"[0.012847400628099172, 0.017999688719008266, 0..."
2,JM12-PM1-MLN10L_950_H1,IGHV2-70*19,CAGGTCACCCTGAGGGAGTCCGGTCCTGCACTAGTAAAGCCCCCCC...,75,CAGGTCACCTTGAGGGAGTCTGGTCCTGCGCTGGTGAAACCCACAC...,0.025991,0.023579,0.011948,0.004347,[CAGGTCACCTTGAAAGAGTCTGGAACCGCACAGGTGAAACCCACA...,"[0.0061951767478991586, 0.01827287725409836, 0..."
3,JM12-PM1-MLN10R_1159_H2,IGHV4-59*02,CAGGTGCAGCTGCAAGAGTCGGGCCCAGGACTGCTGAAGCCTGCGG...,70,CAGGTGCAGCTGCAGGAGTCGGGCCCAGGACTGGTGAAGCCTTCGG...,0.020797,0.025760,0.016321,0.007032,[CATGTACAGCGGCAGATAGCGGGCCCAATACTGGGGAAGCCTTGG...,"[0.00873541197435897, 0.02330020374576271, 0.0..."
4,JM12-PM1-MLN10R_1902_H2,IGHV1-69*04,CAGATCCAGTTGCTACAGTCTGGGCCTGAGGTGAAGAAGCCTGGGT...,67,CAGGTCCAGCTGGTGCAGTCTGGGGCTGAGGTGAAGAAGCCTGGGT...,0.017293,0.025106,0.015888,0.005085,[CAAGACCAGCTGGTGCAGTCTGGGACTGAGGAGAAGAAGACGGGC...,"[0.014508072015873006, 0.024476364915966373, 0..."


In [85]:
# write new file
df.to_csv('./Random/MLN_withCSC_new.csv', index=False)